Initial query

In [1]:
import requests
import json

url = "http://localhost:11434/api/chat"
payload = {
    "model": "llama3.1",
    "messages": [{"role": "user", "content": "Give me some Unsloth.ai News?"}],
    "stream": False
}
response = requests.post(url, data=json.dumps(payload))
print(response.json())

{'model': 'llama3.1', 'created_at': '2025-07-06T23:54:24.498315152Z', 'message': {'role': 'assistant', 'content': 'There is no such thing as "Unsloth.ai" news. It appears to be a made-up or fictional term.\n\nIf you\'re looking for information on AI, technology, or industry-related news, I\'d be happy to provide updates and insights from reputable sources like:\n\n* Google\'s AI research\n* Meta AI (formerly Facebook AI)\n* Microsoft AI\n* OpenAI\n* Research papers and publications in the field of artificial intelligence\n\nLet me know if there\'s something specific you\'re interested in, and I\'ll do my best to help!'}, 'done_reason': 'stop', 'done': True, 'total_duration': 6440074865, 'load_duration': 5076463525, 'prompt_eval_count': 18, 'prompt_eval_duration': 208438417, 'eval_count': 112, 'eval_duration': 1153440250}


Now let's try the same command with Python.

First we need to install the Ollama library for Python

In [2]:
!pip install ollama

Looking in indexes: https://pypi.org/simple/
  Using cached ollama-0.5.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
Using cached ollama-0.5.1-py3-none-any.whl (13 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)


Next let's create a simple python query using the Ollama library

In [3]:
import ollama
response = ollama.chat(
    model="llama3.1",
    messages=[
        {"role": "user", "content": "Give me some Unsloth.ai News?"},
    ],
)
print(response["message"]["content"])

I couldn't find any information on "Unsloth.ai". It's possible that it's a fictional company, or it may be a very new and unknown entity. Can you provide more context or clarify what Unsloth.ai is? I'll do my best to help.

However, if you're interested in news related to AI, machine learning, or technology, I'd be happy to share some general updates or trends with you!


As you can see unsloth.ai causes some hallucinations with llama3.1 and the results can be quite humorous. We will want to fix this by fine tuning the model and give it some information about the Unsloth project. 

The first step in doing this is to convert the provided unsloth_documentation.pdf into chunks using the PyPDFLoader library from LangChain. To use LangChain we will need to install the dependencies.

In [5]:
!pip install langchain langchain_community pypdf

Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 12.1 MB/s eta 0:00:00


With everything installed we can split our PDF into chunks

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("unsloth_documentation.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, 
            chunk_overlap=50
        )
split_docs = text_splitter.split_documents(docs)
print(split_docs[0])

page_content='Finetune Gemma 3n, Qwen3, Llama 4, Phi-4 & Mistral 2x faster with
80% less VRAM!
Finetune for Free
Notebooks are beginner friendly. Read our guide. Add your dataset, click “Run
All”, and export your finetuned model to GGUF, Ollama, vLLM or Hugging
Face.
Unsloth supports Free Notebooks Performance Memory use
Gemma 3n (4B) Start for free 1.5x faster 50% less
Qwen3 (14B) Start for free 2x faster 70% less
Qwen3 (4B):
GRPO
Start for free 2x faster 80% less
Gemma 3 (4B) Start for free 1.6x faster 60% less
Llama 3.2 (3B) Start for free 2x faster 70% less
Phi-4 (14B) Start for free 2x faster 70% less
Llama 3.2 Vision
(11B)
Start for free 2x faster 50% less
Llama 3.1 (8B) Start for free 2x faster 70% less
Mistral v0.3 (7B) Start for free 2.2x faster 75% less
Orpheus-TTS
(3B)
Start for free 1.5x faster 50% less
• See all our notebooks for: Kaggle, GRPO,TTS & Vision
• See all our models and all our notebooks
• See detailed documentation for Unsloth here
Quickstart' metadata={'produc

Now that we have the PDF chunks we can use ollama to generate data for our fine tuning

In [28]:
import json
from typing import List 
from pydantic import BaseModel
import ollama

def prompt_template(data: str, num_records: int = 5):

    return f"""You are an expert data curator assisting a machine learning engineer in creating a high-quality instruction tuning dataset. Your task is to transform 
    the provided data chunk into diverse question and answer (Q&A) pairs that will be used to fine-tune a language model. 

    For each of the {num_records} entries, generate one or two well-structured questions that reflect different aspects of the information in the chunk. 
    Ensure a mix of longer and shorter questions, with shorter ones typically containing 1-2 sentences and longer ones spanning up to 3-4 sentences. Each 
    Q&A pair should be concise yet informative, capturing key insights from the data.

    Structure your output in JSON format, where each object contains 'question' and 'answer' fields. The JSON structure should look like this:

        "question": "Your question here...",
        "answer": "Your answer here..."

    Focus on creating clear, relevant, and varied questions that encourage the model to learn from diverse perspectives. Avoid any sensitive or biased 
    content, ensuring answers are accurate and neutral.

    Example:
    
        "question": "What is the primary purpose of this dataset?",
        "answer": "This dataset serves as training data for fine-tuning a language model."
    

    By following these guidelines, you'll contribute to a robust and effective dataset that enhances the model's performance."

    ---

    **Explanation:**

    - **Clarity and Specificity:** The revised prompt clearly defines the role of the assistant and the importance of the task, ensuring alignment with the 
    project goals.
    - **Quality Standards:** It emphasizes the need for well-formulated Q&A pairs, specifying the structure and content of each question and answer.
    - **Output Format:** An example JSON structure is provided to guide the format accurately.
    - **Constraints and Biases:** A note on avoiding sensitive or biased content ensures ethical considerations are met.
    - **Step-by-Step Guidance:** The prompt breaks down the task into manageable steps, making it easier for the assistant to follow.

    This approach ensures that the generated data is both high-quality and meets the specific requirements of the machine learning project.
    
    Data
    {data}
    """

class Record(BaseModel):
    question: str
    answer: str

class Response(BaseModel):
    generated: List[Record]

def llm_call(data: str, num_records: int = 5) -> dict:
    response = ollama.generate(
        model="llama3.1",
        prompt=prompt_template(data, num_records),
        options={
            "num_predict": 2000
        },
        format=Response.model_json_schema(),
    )
    return json.loads(response['response'])

dataset = []
for i, chunk in enumerate(split_docs):
    data = llm_call(chunk)
    print(data)
    for pair in data['generated']:
        print(pair)
        dataset.append({
                'question': pair['question'],
                'answer': pair['answer']
            })
tuning_data = 'unsloth_data.json'
with open(tuning_data,'w') as f: 
    json.dump(dataset, f) 

print(f"Done writing {tuning_data} to system")

{'generated': [{'question': "What are the benefits of using Unsloth's Notebooks for fine-tuning language models?", 'answer': "Unsloth's Notebooks offer a faster and more memory-efficient way to finetune language models, with some models being 2x faster and requiring up to 80% less VRAM."}, {'question': 'Which Unsloth Notebooks support free usage for beginners?', 'answer': "Unsloth's Notebooks are beginner-friendly, and users can start using them for free, allowing them to finetune their models without any additional cost."}, {'question': 'What is the difference in performance between Gemma 3n and Qwen3 when it comes to fine-tuning language models?', 'answer': 'Gemma 3n offers 1.5x faster performance compared to Qwen3, while also requiring 50% less VRAM.'}, {'question': "Can I export my finetuned model to various platforms using Unsloth's Notebooks?", 'answer': "Yes, users can export their finetuned models to GGUF, Ollama, vLLM, or Hugging Face using Unsloth's Notebooks."}, {'question':

Now that we have a set of training data we are ready to train!

Before doing however we need to install unsloth to help speedup the performance the reduce the RAM required for training.

In [29]:
!pip install unsloth vllm

Looking in indexes: https://pypi.org/simple/
  Using cached unsloth-2025.6.12-py3-none-any.whl.metadata (46 kB)
  Using cached vllm-0.9.1-cp38-abi3-manylinux1_x86_64.whl.metadata (15 kB)
  Using cached unsloth_zoo-2025.6.8-py3-none-any.whl.metadata (8.1 kB)
  Using cached torch-2.7.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached xformers-0.0.31-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached triton-3.3.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.5 kB)
  Using cached tyro-0.9.26-py3-none-any.whl.metadata (12 kB)
  Using cached transformers-4.53.1-py3-none-any.whl.metadata (40 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached 

You will also need PyTorch

In [31]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128


Validate the the device is found.

In [40]:
import torch
use_cuda = torch.cuda.is_available()
print("CUDA available:", use_cuda)
if use_cuda:
    device = torch.device("cuda")
    print(f"Using device: {device}")
    print(torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using device: CPU")

CUDA available: True
Using device: cuda
NVIDIA GeForce RTX 5070


How let's setup our tokenizer using unsloth

In [36]:
from unsloth import FastLanguageModel

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None
)

prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
"You are a helpful, honest and harmless assitant designed to help engineers. Think through each question logically and provide an answer. Don't make things up, if you're unable to answer a question advise the user that you're unable to answer as it is outside of your scope.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("data", split='train')
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
print(dataset["text"][0])

model = FastLanguageModel.get_peft_model(
    model,
    r=128,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=256,
    lora_dropout=0, 
    bias="none", 
   
    use_gradient_checkpointing="unsloth", 
    random_state=3407,
    use_rslora=False, 
    loftq_config=None,
)

==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.1. vLLM: 0.9.1.
   \\   /|    NVIDIA GeForce RTX 5070. Num GPUs = 1. Max memory: 11.94 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 12.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Now that we have the tokenizer we can setup the prompt for fine tuning.

In [33]:

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    num_train_epochs = 10,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

NameError: name 'model' is not defined

The next step is to map our dataset to the template

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 117 | Num Epochs = 25 | Total steps = 200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 335,544,320 of 8,000,000,000 (4.19% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.546100
2,0.472800
3,0.095300
4,0.058600
5,0.042600
6,0.024500
7,0.037500
8,0.000300
9,0.003900
10,0.000100


549.5646 seconds used for training.
9.16 minutes used for training.
Peak reserved memory = 10.236 GB.
Peak reserved memory for training = 3.576 GB.
Peak reserved memory % of max memory = 85.729 %.
Peak reserved memory for training % of max memory = 29.95 %.


Save the new model

In [9]:
new_model_local = "Llama-3.1-8B-unsloth"
model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local) # Local saving

('Llama-3.1-8B-unsloth/tokenizer_config.json',
 'Llama-3.1-8B-unsloth/special_tokens_map.json',
 'Llama-3.1-8B-unsloth/tokenizer.json')

host in local ollama

In [11]:
!ollama create unsloth-trained

gathering model components ⠙ 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


gathering model components ⠹ gathering model components ⠹ gathering model components 
copying file sha256:72cc873cb9a73ac730a3fd3680d4c2380d9ad7cb402bbafc8c05e0d441276675 0% ⠋ 
copying file sha256:36f0351f1afd665ae72623dc7e0d7f51272123f992e93ddda1cb8b625c062778 100% 
copying file sha256:52716f60c3ad328509fa37cdded9a2f1196ecae463f5480f5d38c66a25e7a7dc 100% 
copying file sha256:32f404d626cf7b1b6eea36c241ae7cbd6ec29c777a8701ea48c0fe9ebb94c9b1 100% 
copying file sha256:889397283200148c6aabc316131bb48d62f4b547e4e53da6e13b31851f6a01a1 100% gathering model components 
copying file sha256:72cc873cb9a73ac730a3fd3680d4c2380d9ad7cb402bbafc8c05e0d441276675 5% ⠙ 
copying file sha256:36f0351f1afd665ae72623dc7e0d7f51272123f992e93ddda1cb8b625c062778 100% 
copying file sha256:52716f60c3ad328509fa37cdded9a2f1196ecae463f5480f5d38c66a25e7a7dc 100% 
copying file sha256:32f404d626cf7b1b6eea36c241ae7cbd6ec29c777a8701ea48c0fe9ebb94c9b1 100% 
copying file sha256:889397283200148c6aabc316131bb48d62f4b547e4e53da6

Call latest model from ollama

In [12]:
import ollama
response = ollama.chat(
    model="unsloth-trained",
    messages=[
        {"role": "user", "content": "Give me some Unsloth.ai News?"},
    ],
)
print(response["message"]["content"])

I'm not aware of any recent news or updates from Unsloth.ai. As a conversational AI, I don't have direct access to the web and may not be up-to-date on the latest developments.

However, I can suggest some possible ways to find recent news or updates from Unsloth.ai:

1. Check their website: You can visit Unsloth.ai's website and check for any recent blog posts, announcements, or updates.
2. Social media: Unsloth.ai may have a social media presence on platforms like Twitter, LinkedIn, or Facebook. You can try searching for them on these platforms to see if they've posted any recent updates.
3. News articles: You can try searching online for news articles related to Unsloth.ai or their products.

If you're looking for specific information, I'd be happy to help with that!


Results inconsistent let's try rag

In [20]:
import ollama
from PyPDF2 import PdfReader
import os

# Load and extract text from the PDF
def load_pdf_content(pdf_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF file not found: {pdf_path}")
    
    reader = PdfReader(pdf_path)
    text_content = ""
    for page in reader.pages:
        text_content += page.extract_text() + "\n"
    return text_content

# Load the unsloth documentation
pdf_content = load_pdf_content("unsloth_documentation.pdf")

# Create the RAG-enhanced prompt
rag_prompt = f"""Based on the following documentation about Unsloth.ai, please answer the user's question:

Documentation:
{pdf_content}

User Question: Give me some Unsloth.ai News?

Please provide a comprehensive answer based on the documentation provid d."""

response = ollama.chat(
    model="llama3.1",
    messages=[
        {"role": "user", "content": rag_prompt},
    ],
)
print(response["message"]["content"])

Based on the provided documentation, here are some key points and news related to Unsloth:

**News and Updates**

* Unsloth was tested using the Alpaca Dataset with a batch size of 2, gradient accumulation steps of 4, rank = 32, and applied QLoRA on all linear layers (q, k, v, o, gate, up, down). The results showed that Unsloth achieves a significant VRAM reduction of >75% compared to Hugging Face + FA2.
* Benchmarking of Unsloth was also conducted by Hugging Face. They tested Llama 3.1 (8B) and Llama 3.3 (70B) with QLoRA on all linear layers and reported that Unsloth achieves a longer context length than Hugging Face + FA2.

**Recent Developments**

* Unsloth has added support for 4-bit quantization, which reduces memory usage while maintaining accuracy.
* The team has also implemented full finetuning capabilities, allowing users to fine-tune models from scratch.
* A new LoRA module called "unsloth" has been introduced, which uses 30% less VRAM and fits 2x larger batch sizes.

**Recen